In [2]:
import numpy as np
import pandas as pd
import random
import json
import agent
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

In [ ]:
with open("./release_evidences.json", 'r')as file:
    data = json.load(file)

In [4]:
data

{'E_91': {'name': 'E_91',
  'code_question': 'E_91',
  'question_fr': 'Avez-vous objectivé ou ressenti de la fièvre?',
  'question_en': 'Do you have a fever (either felt or measured with a thermometer)?',
  'is_antecedent': False,
  'default_value': 0,
  'value_meaning': {},
  'possible-values': [],
  'data_type': 'B'},
 'E_55': {'name': 'E_55',
  'code_question': 'E_53',
  'question_fr': 'Avez-vous de la douleur quelque part?',
  'question_en': 'Do you feel pain somewhere?',
  'is_antecedent': False,
  'default_value': 'V_123',
  'value_meaning': {'V_123': {'fr': 'nulle part', 'en': 'nowhere'},
   'V_14': {'fr': 'aile iliaque(D)', 'en': 'iliac wing(R)'},
   'V_15': {'fr': 'aile iliaque(G)', 'en': 'iliac wing(L)'},
   'V_16': {'fr': 'aine(D)', 'en': 'groin(R)'},
   'V_17': {'fr': 'aine(G)', 'en': 'groin(L)'},
   'V_19': {'fr': 'aisselle(G)', 'en': 'axilla(L)'},
   'V_18': {'fr': 'aisselle(D)', 'en': 'axilla(R)'},
   'V_20': {'fr': 'amygdale(D)', 'en': 'tonsil(R)'},
   'V_21': {'fr': 'a

In [5]:
new_dict = {}
for i in data:
    for quest in data[i]:
        if quest == 'question_en':
            question = data[i][quest]
            new_dict[i] = question
            break

In [12]:
df = pd.DataFrame(list(new_dict.items()), columns=['Code', 'Question'])
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])


,Code,Question
0,E_91,Do you have a fever (either felt or measured with a thermometer)?
1,E_55,Do you feel pain somewhere?
2,E_53,"Do you have pain somewhere, related to your reason for consulting?"
3,E_57,Does the pain radiate to another location?
4,E_54,Characterize your pain:
5,E_59,How fast did the pain appear?
6,E_56,How intense is the pain?
7,E_58,How precisely is the pain located?
8,E_159,Did you lose consciousness?
9,E_133,Where is the affected region located?


In [7]:
import json

class Interact_Environment:
    def __init__(self, args, evidence_filepath, condition_filepath):
        # Initialize environment with evidence and condition files
        self.args = args
        self.evidence_data = self.load_json(evidence_filepath)  # Load evidence data from JSON
        self.condition_data = self.load_json(condition_filepath)  # Load condition data from JSON
        self.state = self.initialize_state()  # Initialize state (can be an empty dict, list, etc.)
        self.done = False
        self.turn = 0

    def load_json(self, filepath):
        """Loads JSON data from a file."""
        with open(filepath, 'r') as file:
            data = json.load(file)
        return data

    def initialize_state(self):
        """Initializes the state of the environment."""
        return {
            'evidences': {},  # Store evidence gathered (e.g., symptoms, antecedents)
            'diagnosis': None,  # Track the current diagnosis (if applicable)
        }

    def reset(self):
        """Resets the environment to the initial state."""
        self.state = self.initialize_state()
        self.done = False
        self.turn = 0
        return self.state

    def step(self, action):
        """Process the action (ask a question) and update the environment's state."""
        print(f"Action received: {action}")  # Debugging: Print the action
        evidence_code = f"E_{action}"  # Assuming action is an integer or a string of form '30', '65', etc.

        # Check if the action corresponds to a valid evidence code
        if evidence_code not in self.evidence_data:
            raise KeyError(f"Evidence code {evidence_code} not found in evidence_data.")  # Debugging

        evidence = self.evidence_data[evidence_code]
        evidence_type = evidence['data_type']
        
        # Handle different types of evidence (binary, multi-choice, categorical)
        if evidence_type == 'B':
            answer = self.ask_binary_question(evidence)  # Assume yes/no answer
        elif evidence_type == 'M':
            answer = self.ask_multi_choice_question(evidence)  # Multi-choice handling
        elif evidence_type == 'C':
            answer = self.ask_categorical_question(evidence)  # Categorical handling

        # Update the state with the answer
        next_state = self.update_state(answer)
        reward = self.calculate_reward(next_state)
        done = self.check_done(next_state)

        return next_state, reward, done

    def ask_binary_question(self, evidence):
        """Handles binary questions (yes/no)."""
        print(f"Asking binary question: {evidence['question_en']}")
        # Assume yes/no answer (for simulation, user input can be added)
        return 1  # Example: assume user answers "Yes"

    def ask_multi_choice_question(self, evidence):
        """Handles multi-choice questions."""
        print(f"Asking multi-choice question: {evidence['question_en']}")
        # Simulate asking multi-choice question and getting an answer
        return "V_107"  # Example: assume user selects one of the possible values

    def ask_categorical_question(self, evidence):
        """Handles categorical questions."""
        print(f"Asking categorical question: {evidence['question_en']}")
        # Simulate asking categorical question and getting an answer
        return "V_86"  # Example: assume user selects one of the possible values

    def update_state(self, answer):
        """Updates the state based on the provided answer."""
        new_state = self.state.copy()
        new_state['evidences'][answer] = True  # Store the evidence as true (meaning it was asked)
        return new_state

    def calculate_reward(self, next_state):
        """Calculates a reward based on the current state (for learning)."""
        # Placeholder: reward could be based on how many evidences match the conditions
        return 1  # Simple placeholder reward

    def check_done(self, next_state):
        """Checks if the environment's task is complete (e.g., enough evidence gathered)."""
        # Placeholder condition: the task is done if 10 turns have passed
        if self.turn >= self.args["MAXSTEP"]:
            self.done = True
        return self.done


In [8]:

CUDA_device = 0
class sym_acquire_func(nn.Module):
    """docstring for Net"""
    def __init__(self, state_size, action_size):
        super(sym_acquire_func, self).__init__()
        self.fc1 = nn.Linear(state_size, 1024*2)
        self.fc2 = nn.Linear(1024*2, 2048*1)
        self.fc3 = nn.Linear(2048*1, 1024*2)
        self.out = nn.Linear(1024*2, action_size)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        action_prob = F.softmax(self.out(x), dim = 1)

        return action_prob
    
class diagnosis_func(nn.Module):
    """docstring for Net"""
    def __init__(self, state_size, disease_size):
        super(diagnosis_func, self).__init__()
        self.fc1 = nn.Linear(state_size, 1024*2)
        self.fc2 = nn.Linear(1024*2, 1024*2)
        self.out = nn.Linear(1024*2, disease_size)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        output = self.out(x)
        return output
    
class CompleteModel:
    def __init__(self, state_size, disease_size, symptom_size, policy_path=None, classifier_path=None):
        # Initialize the model with given parameters
        self.state_size = state_size
        self.disease_size = disease_size
        self.symptom_size = symptom_size
        self.policy_path = policy_path
        self.classifier_path = classifier_path
        self.model = self.load_model()  # Load the model (e.g., policy, classifier)

    def load_model(self):
        """Loads the pre-trained model (policy, classifier)."""
        # Placeholder for model loading, could be neural networks, classifiers, etc.
        print("Loading model...")
        return {}

    def choose_action_s(self, state):
        """Choose an action based on the current state."""
        # Simulate choosing an action: pick a random symptom/evidence code
        print("Choosing action...")
        available_actions = list(state['evidences'].keys())  # Example: get all possible actions
        action = random.choice(available_actions) if available_actions else "30"  # Default action if no evidence
        return action

    def choose_diagnosis(self, state):
        """Choose a diagnosis based on the gathered evidence."""
        # Placeholder: diagnose based on evidence
        print("Choosing diagnosis...")
        # Placeholder diagnosis logic
        diagnosis = "Myasthenia gravis"  # Example diagnosis
        probabilities = [random.random() for _ in range(self.disease_size)]  # Random probabilities
        return diagnosis, probabilities

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [10]:
policy_path = 'D:\DDXPlus\ddxplus\code\\aarlc\ddxplus_code\output\\best_policy_casande_1_1.0_2.826_1.pth'
classifier_path = 'D:\DDXPlus\ddxplus\code\\aarlc\ddxplus_code\output\\best_classifier_casande_1_1.0_2.826_1.pth'
evidence_filepath = "D:/DDXPlus/ddxplus/code/dataset/release_evidences.json"
condition_filepath = "D:/DDXPlus/ddxplus/code/dataset/release_conditions.json"

In [11]:
args = {"MAXSTEP": 10}
env = Interact_Environment(args, evidence_filepath, condition_filepath)

env = Interact_Environment(args, evidence_filepath=evidence_filepath, condition_filepath=condition_filepath)

# Initialize model
state_size = 922  # Example state size, adjust based on your actual state size
disease_size = 49  # Example number of diseases
symptom_size = 223  # Example number of symptoms
model = CompleteModel(state_size=state_size, disease_size=disease_size, symptom_size=symptom_size)

# Start the interaction loop
state = env.reset()
done = False
turn = 0
while not done and turn < args["MAXSTEP"]:
    print(turn)
    action = model.choose_action_s(state)  # Agent chooses an action (ask a question)
    
    # Perform the action in the environment (ask the patient for a symptom)
    next_state, reward, done = env.step(action)
    
    # Optionally, diagnose if enough evidence has been gathered
    if turn > 2:  # Example condition where enough data is gathered
        diagnosis, probabilities = model.choose_diagnosis(next_state)
        print(f"Diagnosis: {diagnosis}, Probabilities: {probabilities}")
    
    # Update the state for the next turn
    state = next_state
    turn += 1
    print(f"Turn: {turn}, Action: {action}, Reward: {reward}")

Loading model...
0
Choosing action...
Action received: 30
Asking binary question: Do you feel your abdomen is bloated or distended (swollen due to pressure from inside)?
Turn: 1, Action: 30, Reward: 1
1
Choosing action...
Action received: 1
Asking binary question: Are you currently being treated or have you recently been treated with an oral antibiotic for an ear infection?
Turn: 2, Action: 1, Reward: 1
2
Choosing action...
Action received: 1
Asking binary question: Are you currently being treated or have you recently been treated with an oral antibiotic for an ear infection?
Turn: 3, Action: 1, Reward: 1
3
Choosing action...
Action received: 1
Asking binary question: Are you currently being treated or have you recently been treated with an oral antibiotic for an ear infection?
Choosing diagnosis...
Diagnosis: Myasthenia gravis, Probabilities: [0.09297961588435988, 0.3524394336515946, 0.5730491179202065, 0.02752957397490763, 0.18286359225536974, 0.023697607265578347, 0.7640145517719428